In [417]:
from collections import Counter

import numpy as np

data = np.loadtxt("data.csv", delimiter=",")
labels = np.loadtxt("label.csv").astype(int)

In [418]:
def euclideanDistance(x,y):
    distance = np.sqrt(np.sum((x - y) ** 2))
    return distance

In [419]:
def cosineDistance(x,y):
    denominator = (np.linalg.norm(x) * np.linalg.norm(y))

    if (denominator == 0):
        return 1
    else:
        return (1 - (np.dot(x,y)))/denominator

In [420]:
def jaccardDistance(x,y):
    min = np.sum(np.minimum(x,y))
    max = np.sum(np.maximum(x,y))

    if(max == 0):
        return 0
    else:
        return 1 - (min/max)

In [421]:
def SSE(clusters, centroids):
    error = 0
    for i, cluster in enumerate(clusters):
        for index in cluster:
            if len(cluster) > 0:
                error += np.square(jaccardDistance(data[index], centroids[i]))
    return error #/ (data.shape[0] * data.shape[1])
#The above / (data.shape...) is for standardization of Euclidean distance since it ends up giving a way larger number than the other SSEs so that I can compare them easier.

In [422]:
def clusterLabels(clusters):
    returnLabels = []

    for cluster in clusters:
        clusterCounts = []
        for index in cluster:
            clusterCounts.append(labels[index])

        majority = Counter(clusterCounts).most_common(1)[0][0]
        returnLabels.append(majority)

    return returnLabels

In [423]:
def predictedLabels(returnLabels, centroids):
    predictions = []
    for x in data:
        dists = [jaccardDistance(x, c) for c in centroids]
        cluster_index = np.argmin(dists)
        predictions.append(returnLabels[cluster_index])
    return np.array(predictions)

In [424]:
def accuracy(pred, true):
    return np.mean(pred == true)

In [425]:
def kmeans(k=10, iterations=100, seed=0):
    np.random.seed(seed)
    samples, features = data.shape
    totalIterations = 0
    previousSSE = float("inf")

    indices = np.random.choice(samples, k, replace=False)
    centroids = data[indices].copy()
    clusters = []

    for blank in range(iterations):
        clusters = [[] for i in range(k)]
        for i in range(samples):
            distances = []
            for c in centroids:
                distances.append(jaccardDistance(data[i], c))
            clusters[np.argmin(distances)].append(i)

        newCentroids = np.zeros_like(centroids)
        for i in range(k):
            if clusters[i]:
                newCentroids[i] = np.mean(data[clusters[i]], axis=0)
            else:
                newCentroids[i] = centroids[i]

        #Get rid of for Q4 when needed.
        #if np.allclose(centroids, newCentroids):
            #break

        #Below is just for Q3 and also for Q4 when needed.
        #if previousSSE < SSE(clusters, centroids):
            #break
        #else:
            #previousSSE = SSE(clusters, centroids)

        centroids = newCentroids
        totalIterations += 1

    return clusters, centroids, totalIterations

In [426]:
##Testing for Questions
##Q1 - went up to euclideanDistance and switched it for the other 2. also in SSE function.

#clusters, centroids, iterations = kmeans()

#SSE(clusters, centroids)

First value was 25321064456.818565 for Euclidean, but this also makes sense since it is not bounded the same way cosine and jaccard are. If I try to standardize it to be in align with the other two SSEs by diving by the total number of features * observations, I get 3229.7276092880825.
Second was 5745.707845814075 for cosine
Third was 3690.822622100952 for Jaccard

This would make Jaccard the best method by going off of just normal SSEs like the question asked, however if we standardize Euclidean for easier comparison, it would have the best SSE.

In [427]:
##Q2 - same as Q1 where i just switch out where I run Euclidean distance for the other distance measurments.

#clusters, centroids, iterations = kmeans()
#returnLabels = clusterLabels(clusters)
#predictLabels = predictedLabels(returnLabels, centroids)
#accuracy(predictLabels, labels)

For Euclidean distance, the accuracy is .5977.
For Cosine distance the accuracy was 0.5478.
For Jaccard, the accuracy was 0.5443.

The best metric here was the Euclidean-k-means for accuracy.

In [428]:
##Q3 - same as Q1-2 where I just change the distance measurement in all spots for whichever one I am running. Also line 29 in kmeans function needs to be done for this to work as well.

#clusters, centroids, iterations = kmeans()
#iterations

For Jaccard, I got 11 iterations before it stopped, and it stopped via SSE value increasing in the next iteration.
For cosine, it stopped after 1 iteration due to the SSE value increasing in the next iteration.
For Euclidean it stopped after 47 iterations due to converging.

I believe that this is due to the algorithmic way that SSE is computed for cosine distance and jaccard distance. When I ran without the SSE value increase in the next iteration I got 45 for cosine and 54 for jaccard.

Still with all of the methods in place, Euclidean would take the most number of iterations and time to converge.

In [429]:
##Q4 - Similar to Q3 but I have more stopping points that I selected inside of kmeans that I have marked above.

clusters, centroids, iterations = kmeans()

SSE(clusters, centroids)


np.float64(3690.822622100952)

when there is no change in centroid position
Euclidean: 3229.727
Cosine: 5745.707
Jaccard: 3690.822
when the SSE value increases in the next iteration
Euclidean: 3229.727
Cosine: 5427.275
Jaccard: 3686.075
when the maximum preset value (which is 100) of iteration is complete
Euclidean: 3229.727
Cosine: 5745.707
Jaccard: 3690.822

Above, the values for Euclidean distance all make sense. Once there is no change in centroid position, there isn't going to be many changes in the SSE. Since converging happens first, the other values will be the same. For Cosine and Jaccard, because the values for SSE are computed based on a fixed scale, the lowest and highest values won't be that different. Also it is possible for the SSE values to go up and down for those two, the will have different values compared to the max iterations and when there is no change.

Q5:
My observations were that first SSE is not the easiest to compare amongst the several different ways of K-means. I am not quite sure that it is a great method for determining which method is the best. Accuracy and the time/number of iterations seems best to me. The total number of iterations only was changed significantly when we stopped based off of SSE values. Outside of that it seems that we were able to have more consistent data when focusing on the other criteria. When we mainly focus on that, it seems that Euclidean performs the best with the other two performing slightly worse.
Additionally, I noticed that when trying to label each cluster. that some clusters were assigned to the same centroid. This would cause accuracies to go down a bit. There are some ways that I could prevent this though like taught in class such as having more clusters and then picking the ones that are the furthest away from each other.